In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import csv

def save_page_html(url, save_path):
    chrome_options = Options()
    chrome_options.add_argument("--incognito")

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)

    # Ottieni l'HTML della pagina
    page_html = driver.page_source

    # Salva l'HTML su un file
    with open(save_path, "w", encoding="utf-8") as file:
        file.write(page_html)

    print(f"HTML salvato su {save_path}")

    # Chiudi la finestra del browser
    driver.quit()

# Esempio di utilizzo
website_url = "https://www.snai.it/sport/CALCIO/SERIE%20A"
output_path = "last_odds.txt"
save_page_html(website_url, output_path)


HTML salvato su last_odds.txt


In [44]:
def filter_words(input_file, output_file, start_date, end_date):
    with open(input_file, 'r', encoding='utf-8') as infile:
        words = infile.read().split()

    start_index = next((i for i, word in enumerate(words) if word == start_date), None)
    end_index = next((i for i, word in enumerate(words) if word == end_date), None)

    # Verifica se le parole '15/12' e '18/12' sono presenti nel file
    if start_index is None or end_index is None:
        print("Le date di inizio o fine non sono presenti nel file.")
        return

    # Filtra le parole tra '15/12' e '18/12' inclusi
    filtered_words = words[start_index:end_index + 20644]

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write(' '.join(filtered_words))

# Esempio di utilizzo
input_file_path = "last_odds.txt"
output_file_path = "last_odds.txt"
start_date_filter = '15/12'
end_date_filter = '18/12'

filter_words(input_file_path, output_file_path, start_date_filter, end_date_filter)


In [71]:
def find_numbers_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Definisci il pattern regex per trovare le stringhe '\d\d,\d\d' o '\d,\d\d'
    pattern = re.compile(r'\b\d{1,2},\d{1,2}\b')

    # Trova tutte le corrispondenze nel testo
    matches = pattern.findall(content)

    return matches

# Esempio di utilizzo
file_path = "last_odds.txt"
result = find_numbers_in_file(file_path)

if len(result) != 70:
    print(' WARNING, LESS THAN 10 MATCHES SELECTED ')

with open('last_odds.csv', mode='w', newline='', encoding='utf-8',) as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['1','x','2'])
    reset=0
    temp=[]
    for numero in result:
        temp.append(numero.replace(',', '.'))
        reset=reset+1
        if (reset % 7 ==0):
            csv_writer.writerow(temp[::-1][:3])
            print(temp)
            temp=[]
            reset=0


70
['1.65', '2.10', '2.35', '1.50', '1.85', '3.25', '4.75']
['1.95', '1.77', '1.95', '1.75', '3.60', '3.35', '2.05']
['1.80', '1.90', '1.53', '2.35', '9.00', '5.50', '1.30']
['1.77', '1.95', '1.90', '1.80', '6.00', '4.00', '1.55']
['1.97', '1.75', '1.70', '2.05', '5.75', '4.25', '1.55']
['1.87', '1.85', '1.70', '2.05', '6.50', '4.50', '1.45']
['2.35', '1.53', '1.65', '2.10', '3.05', '3.50', '2.25']
['1.73', '2.00', '2.40', '1.50', '3.30', '3.00', '2.35']
['1.90', '1.80', '1.92', '1.80', '1.73', '3.70', '4.75']
['1.80', '1.90', '1.55', '2.30', '8.75', '5.50', '1.30']


In [72]:
column_types = {'1': float, 'x': float, '2': float}
last_odds = pd.read_csv(r'last_odds.csv', dtype=column_types)
min_odds = pd.read_csv(r'min_odds.csv', dtype=column_types)

# Creazione del terzo DataFrame contenente il minimo dei valori
df_min = pd.DataFrame()

# Itera attraverso le colonne e le righe per ottenere i minimi
for col in last_odds.columns:
    df_min[col] = [min(a, b) for a, b in zip(min_odds[col], last_odds[col])]

df_min.to_csv('min_odds.csv',index=False)

In [74]:
last_odds + min_odds

,1,x,2
0,9.5,6.5,3.70
1,4.1,6.7,7.20
2,2.6,11.0,18.00
3,3.1,8.0,12.00
4,3.1,8.5,11.50
5,2.9,9.0,13.00
6,4.5,7.0,6.10
7,4.7,6.0,6.60
8,9.5,7.4,3.46
9,2.6,11.0,17.50
